In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import os
import openai
import time
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("BnPC_val_2_label.csv")
df.head()

,sentence1,sentence2,label
0,রায়ের পর হাসছিলেন রিফাত ফরাজীরা,আদালত থেকে আর বাড়ি যাওয়া হলো না মিন্নির,0
1,রায়ের পর হাসছিলেন রিফাত ফরাজীরা,রিফাত হত্যায় স্ত্রী মিন্নিসহ ৬ জনের ফাঁসির রায়,0
2,রায়ের পর হাসছিলেন রিফাত ফরাজীরা,মিন্নির প্রতি অবিচার করা হয়েছে : বাবা,0
3,রায়ের পর হাসছিলেন রিফাত ফরাজীরা,রিফাত হত্যা মামলা : কিছুক্ষণের মধ্যেই রায়,0
4,রায়ের পর হাসছিলেন রিফাত ফরাজীরা,আদালতে এসেছেন বিচারক মো. আছাদুজ্জামান,0


In [ ]:
len(df)

1474

In [ ]:
tokens = [
  "Provide a list of tokens, as there is a limit of 3 requests per minute and 200 request per day per account"
]

In [ ]:
from google.colab import files

df = pd.read_csv("BnPC_val_2_label.csv")
gpt_label = []
gpt_answer = []
start_index = 1238
stop_index = 1394
for i in range(start_index,stop_index+1,7):
  try:
    for j in range(7):
      sent1 = df['sentence1'][i+j]
      sent2 = df['sentence2'][i+j]
      openai.api_key = tokens[j]
      response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
          {
            "role": "system",
            "content": f'''Only reply with \"paraphrase\" or \"non-paraphrase\". You will be given some setence pairs as \"Sentence 1\" and \"Sentence 2\" Determine if
            following sentences are paraphrases or not:\n\n1. in the \"label\".
            Here are some examples:
            Sentence 1: "ফেসবুকে অস্ত্র হাতে ছবি, আলোচনায় এমপি বাবলু",
            Sentence 2: "ফেসবুকে পিস্তল হাতে বগুড়ার এমপি বাবলু",
            label: "paraphrase"

            Sentence 1: "সাংবাদিক রুহুল আমিন গাজী গ্রেফতার",
            Sentence 2: "জামিন মেলেনি, কারাগারে সাংবাদিক রুহুল আমিন",
            label: "paraphrase"

            Sentence 1: "অতিরিক্ত এটর্নি জেনারেল মমতাজ উদ্দিনের পদত্যাগ",
            Sentence 2: "মৌলভীবাজার জুড়ে উচ্ছ্বাস",
            label: "non-paraphrase"

            Sentence 1: "কাদের মোল্লাকে মুক্তিযুদ্ধের ট্রেনিং প্রাপ্ত বলেছিলেন রুহুল আমীন গাজী",
            Sentence 2: "রুহুল আমিন গাজী কারাগারে",
            label: "non-paraphrase"
            '''
          },
          {
            "role": "user",
            "content": f"{{\nsent1: \"{sent1}\", \nsent2: \"{sent2}\"\n}}"
          },
          {
          "role": "assistant",
          "content": "Detect if these are paraphrases or not."
          }
        ],
        temperature=0,  # Set temperature to 0 for deterministic output
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
      )

      # Extract the assistant's response
      assistant_response = response.choices[-1].message["content"]
      print(f"{i+j} -  {sent1} and {sent2}: \n")
      print(assistant_response)
      gpt_answer.append(assistant_response)
      # Now, you can analyze the assistant's response to determine if it's paraphrase or non-paraphrase
      if assistant_response!="paraphrase":
          gpt_label.append(0)
      else:
          gpt_label.append(1)
    time.sleep(20)
  except:
    df = pd.DataFrame({f'GPT_3.5_turbo_label_{start_index}_{i-2}': gpt_label})
    df.to_csv(f"gpt_label_{start_index}_to_{i-2}.csv")
    files.download(f"gpt_label_{start_index}_to_{i-2}.csv")

df = pd.DataFrame({f'GPT_3.5_turbo_label_{start_index}_{stop_index}': gpt_label})
df.to_csv(f"gpt_label_{start_index}_to_{stop_index}.csv")
files.download(f"gpt_label_{start_index}_to_{stop_index}.csv")

1238 -  করোনা আক্রান্ত অভিনেতা সৌমিত্র চট্টোপাধ্যায় and করোনায় আক্রান্ত সৌমিত্র চট্টোপাধ্যায় হাসপাতালে ভর্তি: 

paraphrase
1239 -  করোনা আক্রান্ত সৌমিত্র চট্টোপাধ্যায়, ভর্তি বেলভিউ ক্লিনিকে and করোনা আক্রান্ত সৌমিত্র চট্টোপাধ্যায়, ভর্তি বেলভিউ ক্লিনিকে: 

paraphrase
1240 -  করোনা আক্রান্ত সৌমিত্র চট্টোপাধ্যায়, ভর্তি বেলভিউ ক্লিনিকে and করোনায় আক্রান্ত বর্ষীয়ান অভিনেতা সৌমিত্র চট্টোপাধ্যায়, ভর্তি হাসপাতালে: 

paraphrase
1241 -  করোনা আক্রান্ত সৌমিত্র চট্টোপাধ্যায়, ভর্তি বেলভিউ ক্লিনিকে and করোনায় আক্রান্ত সৌমিত্র চট্টোপাধ্যায় হাসপাতালে ভর্তি: 

paraphrase
1242 -  করোনায় আক্রান্ত বর্ষীয়ান অভিনেতা সৌমিত্র চট্টোপাধ্যায়, ভর্তি হাসপাতালে and করোনায় আক্রান্ত সৌমিত্র চট্টোপাধ্যায় হাসপাতালে ভর্তি: 

paraphrase
1243 -  করোনা আক্রান্ত হয়ে হাসপাতালে সৌমিত্র and এবার করোনা আক্রান্ত সৌমিত্র চট্টোপাধ্যায়! ভর্তি করা হল হাসপাতালে: 

paraphrase
1244 -  করোনা আক্রান্ত হয়ে হাসপাতালে সৌমিত্র and করোনায় আক্রান্ত সৌমিত্র চট্টোপাধ্যায়, ভর্তি হাসপাতালে: 

paraphrase
1245 -  করোনা আক্রান্ত হয়ে হাস

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>